In [1]:
import pyspark
sc = pyspark.SparkContext()

# Method for importing lemmatizer

In [2]:
import csv
new_dict = dict()
def readfile(filename):
    count = 0
    with open(filename) as f:
        csvr = csv.reader(f)
        for line in csvr:
            samp = ""
            for i in range(1,len(line)):
                if(line[i] != ''):
                    samp += line[i] + ','
            new_dict[line[0]] = samp

In [3]:
readfile('new_lemmatizer.csv')
text_file = sc.textFile("co_occur_input")
text_file = text_file.filter(lambda x: x is not u'')  #To remove blank lines

In [4]:
def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

In [5]:
def mainReplace(word):
	word = word.replace(",","")
	word = word.replace("!","")
	word = word.replace("?","")
	word = word.replace(":","")
	word = word.replace("#","")
	word = word.replace("\u","")
	return word

# Main bigram method

In [6]:
def mainMethod(line):
    words = line[0]
    pos = line[1]
    list = []
    one = mainReplace(words[0])
    two = mainReplace(words[1])
    if new_dict.has_key(one) and new_dict.has_key(two):    
        a = new_dict[one]
        b = new_dict[two]
        a = a.split(',')
        b = b.split(',')
        a.remove('')
        b.remove('')
        for i in a:
            for j in b:
                list.append(((i,j),pos))
    elif new_dict.has_key(one):
        a = new_dict[one]
        a = a.split(',')
        a.remove('')
        for i in a:
            list.append(((i, two),pos))
    elif new_dict.has_key(two):
        b = new_dict[two]
        b = b.split(',')
        b.remove('')
        for j in b:
            list.append(((one, j),pos))
    else:
        list.append(((one,two),pos))
    return list

# Code to calculate 2 grams

In [12]:
import itertools
rep = {'j':'i', 'v':'u'}
low = text_file.map(lambda line: line.lower())
text = low.map(lambda line : (line.split('>')[0]+">",replace_all(line.split('>')[1].lstrip(),rep).split()))
bigram = text.flatMap(lambda line:  [(pair, line[0]) for pair in itertools.combinations(line[1], 2)])
reduced = bigram.reduceByKey(lambda a,b: a+" "+b)
lemmatize = reduced.flatMap(mainMethod)
lemmatize.saveAsTextFile('paircount')

# Main trigram method

In [8]:
def mainMethod1(line):
    words = line[0]
    pos = line[1]
    list = []
    one = mainReplace(words[0])
    two = mainReplace(words[1])
    three = mainReplace(words[2])
    if new_dict.has_key(one) and new_dict.has_key(two) and new_dict.has_key(three):    
        a = new_dict[one]
        a = a.split(',')
        a.remove('')
        b = new_dict[two]
        b = b.split(',')
        b.remove('')
        c = new_dict[three]
        c = c.split(',')
        c.remove('')
        for i in a:
            for j in b:
                for k in c:
                    list.append(((i,j,k),pos))
    elif new_dict.has_key(one) and new_dict.has_key(two):
        a = new_dict[one]
        a = a.split(',')
        a.remove('')
        b = new_dict[two]
        b = b.split(',')
        b.remove('')
        for i in a:
            for j in b:
                list.append(((i, j, three),pos))
    elif new_dict.has_key(one) and new_dict.has_key(three):
        a = new_dict[one]
        a = a.split(',')
        a.remove('')
        c = new_dict[three]
        c = c.split(',')
        c.remove('')
        for i in a:
            for k in c:
                list.append(((i, two, k),pos))
    elif new_dict.has_key(two) and new_dict.has_key(three):
        b = new_dict[two]
        b = b.split(',')
        b.remove('')
        c = new_dict[three]
        c = c.split(',')
        c.remove('')
        for j in b:
            for k in c:
                list.append(((one,j,k),pos))
    elif new_dict.has_key(one):
        a = new_dict[one]
        a = a.split(',')
        a.remove('')
        for i in a:
            list.append(((i,two,three),pos))
    elif new_dict.has_key(two):
        b = new_dict[two]
        b = b.split(',')
        b.remove('')
        for j in b:
            list.append(((one,j,three),pos))
    elif new_dict.has_key(three):
        c = new_dict[three]
        c = c.split(',')
        c.remove('')
        for k in c:
            list.append(((one,two,k),pos))
    else:
        list.append(((one,two,three),pos))
    return list

# Code to calculate 3 grams

In [13]:
import itertools
rep = {'j':'i', 'v':'u'}
low = text_file.map(lambda line: line.lower())
text = low.map(lambda line : (line.split('>')[0]+">",replace_all(line.split('>')[1].lstrip(),rep).split()))
trigram = text.flatMap(lambda line:  [(triple, line[0]) for triple in itertools.combinations(line[1], 3)])
reduced = trigram.reduceByKey(lambda a,b: a+" "+b)
lemmatize = reduced.flatMap(mainMethod1)
lemmatize.saveAsTextFile('tricount')